In [ ]:
import os
TEST_MODE = os.environ.get("TEST_MODE", "0") == "1"

# Use local wait_for_seldon_client_predict from notebooks folder
if TEST_MODE:
    from wait_for_seldon_client_predict import wait_for_seldon_client_predict

# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

In [6]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


## Serve Single Model

In [7]:
!helm upgrade -i mymodel ../helm-charts/seldon-single-model --set 'model.image=seldonio/mock_classifier:1.9.0-dev' --namespace seldon

Release "mymodel" does not exist. Installing it now.
NAME: mymodel
LAST DEPLOYED: Mon Nov 17 11:56:26 2025
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [8]:
if not TEST_MODE:
    !helm template mymodel ../helm-charts/seldon-single-model --set 'model.image=seldonio/mock_classifier:1.5.0-dev' | pygmentize -l json

---
# Source: seldon-single-model/templates/seldondeployment.json
{
  "kind": "SeldonDeployment",
  "apiVersion": "machinelearning.seldon.io/v1",
  "metadata": {
    "name": "mymodel",
    "namespace": "seldon",
    "labels": {}
  },
  "spec": {
      "name": "mymodel",
      "protocol": "seldon",
    "annotations": {},
    "predictors": [
      {
        "name": "default",
        "graph": {
          "name": "model",
          "type": "MODEL",
        },
        "componentSpecs": [
          {
            "spec": {
              "containers": [
                {
                  "name": "model",
                  "image": "seldonio/mock_classifier:1.5.0-dev",
                  "env": [
                      {
                        "name": "LOG_LEVEL",
                        "value": "INFO"
                      },
                    ],
                  "resources": {"requests":{"memory":"1Mi"}},
                }
              ]
            },
          }
        ],
        "re

In [2]:
!kubectl wait deployment \
  -l seldon-deployment-id=mymodel \
  --for=condition=available \
  --timeout=120s

deployment.apps/mymodel-default-0-model condition met


### Get predictions

In [3]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    deployment_name="mymodel",
    namespace="seldon",
    gateway_endpoint="localhost:8003",
    gateway="ambassador",
)

#### REST Request

In [9]:
if TEST_MODE:
    wait_for_seldon_client_predict(sc, transport="rest");

In [8]:
r = sc.predict(transport="rest")
assert r.success == True

if not TEST_MODE:
    print(r)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.38409108778875034
  }
}

Response:
{'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.07360583001287857]}}, 'meta': {'requestPath': {'model': 'seldonio/mock_classifier:1.9.0-dev'}}}


#### GRPC Request

In [10]:
if TEST_MODE:
    wait_for_seldon_client_predict(sc, transport="grpc");

In [11]:
r = sc.predict(transport="grpc")
assert r.success == True

if not TEST_MODE:
    print(r)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 1], 'values': [0.13932495342922413]}}}
Response:
{'meta': {'requestPath': {'model': 'seldonio/mock_classifier:1.9.0-dev'}}, 'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.05856093916353605]}}}


In [12]:
!helm delete mymodel

release "mymodel" uninstalled


## Serve REST AB Test

In [13]:
!helm upgrade -i myabtest ../helm-charts/seldon-abtest --namespace seldon

Release "myabtest" does not exist. Installing it now.
NAME: myabtest
LAST DEPLOYED: Mon Nov 17 12:04:10 2025
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [14]:
if not TEST_MODE:
    !helm template ../helm-charts/seldon-abtest | pygmentize -l json

---
# Source: seldon-abtest/templates/ab_test_2pods.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
	"labels": {
	    "app": "seldon"
	},
	"name": "release-name"
    },
    "spec": {
	"name": "release-name",
	"predictors": [
	    {
		"name": "default",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
                                "image": "seldonio/mock_classifier:1.19.0-dev",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
		    "metadata":{
			"labels":{
			    "version":"v2"
			}
		    },    
			"spec":{
			    "containers":[
				{
                                "image": "seldonio/mock_classifier:1.19.0-dev",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
	

In [15]:
!kubectl wait deploy \
  -l seldon-deployment-id=myabtest \
  --for=condition=available \
  --timeout=120s

deployment.apps/myabtest-default-0-classifier-1 condition met
deployment.apps/myabtest-default-1-classifier-2 condition met


### Get predictions

In [17]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    deployment_name="myabtest",
    namespace="seldon",
    gateway_endpoint="localhost:8003",
    gateway="ambassador",
)

#### REST Request

In [18]:
if TEST_MODE:
    wait_for_seldon_client_predict(sc, transport="rest");

In [19]:
r = sc.predict(transport="rest")
assert r.success == True

if not TEST_MODE:
    print(r)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.7417747419036318
  }
}

Response:
{'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.10202797042881677]}}, 'meta': {'requestPath': {'classifier-1': 'seldonio/mock_classifier:1.19.0-dev'}}}


#### gRPC Request

In [20]:
if TEST_MODE:
    wait_for_seldon_client_predict(sc, transport="grpc");

In [21]:
r = sc.predict(transport="grpc")
assert r.success == True

if not TEST_MODE:
    print(r)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 1], 'values': [0.7144898304296823]}}}
Response:
{'meta': {'requestPath': {'classifier-1': 'seldonio/mock_classifier:1.19.0-dev'}}, 'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.09955517875601523]}}}


In [22]:
!helm delete myabtest

release "myabtest" uninstalled


## Serve REST Multi-Armed Bandit

In [23]:
!helm upgrade -i mymab ../helm-charts/seldon-mab --namespace seldon

Release "mymab" does not exist. Installing it now.
NAME: mymab
LAST DEPLOYED: Mon Nov 17 12:05:30 2025
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [25]:
if not TEST_MODE:
    !helm template ../helm-charts/seldon-mab | pygmentize -l json

---
# Source: seldon-mab/templates/mab.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
		"labels": {"app":"seldon"},
		"name": "release-name"
    },
    "spec": {
	"name": "release-name",
	"predictors": [
	    {
		"name": "default",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
                                "image": "seldonio/mock_classifier:1.19.0-dev",				
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
			"spec":{
			    "containers":[
				{
                                "image": "seldonio/mock_classifier:1.19.0-dev",								    
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }
			],
			"terminationGracePeriod

In [26]:
!kubectl wait deploy \
  -l seldon-deployment-id=mymab \
  --for=condition=available \
  --timeout=120s

deployment.apps/mymab-default-0-classifier-1 condition met
deployment.apps/mymab-default-1-classifier-2 condition met
deployment.apps/mymab-default-2-eg-router condition met


### Get predictions

In [27]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    deployment_name="mymab",
    namespace="seldon",
    gateway_endpoint="localhost:8003",
    gateway="ambassador",
)

#### REST Request

In [28]:
if TEST_MODE:
    wait_for_seldon_client_predict(sc, transport="rest");

In [29]:
r = sc.predict(transport="rest")
assert r.success == True

if not TEST_MODE:
    print(r)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.27445760751302317
  }
}

Response:
{'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.06647082668141341]}}, 'meta': {'requestPath': {'classifier-2': 'seldonio/mock_classifier:1.19.0-dev'}}}


#### gRPC Request

In [30]:
if TEST_MODE:
    wait_for_seldon_client_predict(sc, transport="grpc");

In [31]:
r = sc.predict(transport="grpc")
assert r.success == True

if not TEST_MODE:
    print(r)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 1], 'values': [0.8881053062975415]}}}
Response:
{'meta': {'requestPath': {'classifier-2': 'seldonio/mock_classifier:1.19.0-dev'}}, 'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.11623662572040293]}}}


In [32]:
!helm delete mymab

release "mymab" uninstalled
